<a href="https://colab.research.google.com/github/sarathpanat/Pneumonia-detection/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chest X-ray classification - Streamlit

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -U ipykernel

In [7]:
!pip install -q streamlit

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

In [83]:
#@title !/ngrok authtoken XXXXXXX { display-mode: "form" }
!/ngrok authtoken xxxxxxxxxxxxxxxxxxx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [92]:
%%writefile img_classification.py
import keras
from PIL import Image, ImageOps
import numpy as np

def teachable_machine_classification(img, weights_file):
    # Load the model
    model = keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 255)

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    #return np.argmax(prediction)
    return prediction.round() # return position of the highest probability

Overwriting img_classification.py


In [93]:
%%writefile app.py
import streamlit as st
from PIL import Image
from img_classification import teachable_machine_classification

st.sidebar.title("About")
st.sidebar.info("This is ML based Pneumonia detection web app, where you can upload X-ray images and predict whether the X-ray is healthy or not from pneumonia. This model is built with CNN.")
st.sidebar.title("Predict chest X-ray (pneumonia) ")
uploaded_file = st.sidebar.file_uploader("Choose a chest X-ray ...", type="jpg")

st.title("Chest X-ray classification using CNN")
st.header("Predicting pneumonia from chest X-ray images")
st.text("Upload a chest X-ray Image for image classification as Normal or pneumonia")

if uploaded_file is not None:
  if st.button('Predict X-ray'):
    showpred = 1
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded MRI.', use_column_width='True')
    st.write("")
    st.write("Classifying...")
    label = teachable_machine_classification(image, '/content/gdrive/MyDrive/Colab Notebooks/my_model.h5')
    if label == 0:
        st.write("The X ray scan is healthy")
    else:
        st.write("The X ray scan has pneumonia")

Overwriting app.py


In [97]:
get_ipython().system_raw('./ngrok http 8501 &')

In [98]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://0621e2dfb755.ngrok.io


In [ ]:
!streamlit run app.py